Manuall install `structlog`

`pip install structlog`

In [1]:
# %load_ext autoreload
# %autoreload 2
import os

os.environ["BASETEN_WEBHOOK_SECRET"] = "your_baseten_webhook_secret"
os.environ["BASETEN_API_KEY"] = "your_baseten_api_key"

In [2]:
# Read the sample data
import json
from flow_judge import EvalInput

with open("sample_data/csr_assistant.json", "r") as f:
    data = json.load(f)

# Create a list of inputs and outputs
inputs_batch = [
    [
        {"query": sample["query"]},
        {"context": sample["context"]},
    ]
    for sample in data
]
outputs_batch = [{"response": sample["response"]} for sample in data]

# Create a list of EvalInput
eval_inputs_batch = [EvalInput(inputs=inputs, output=output) for inputs, output in zip(inputs_batch, outputs_batch)]

WARNING 10-17 17:58:26 _custom_ops.py:18] Failed to import from vllm._C with ModuleNotFoundError("No module named 'vllm._C'")
INFO 10-17 17:58:26 importing.py:10] Triton not installed; certain GPU-related functions will not be available.


In [3]:
from flow_judge.metrics import RESPONSE_FAITHFULNESS_5POINT
from flow_judge.utils.prompt_formatter import format_user_prompt, format_vars, format_rubric

metric = RESPONSE_FAITHFULNESS_5POINT

def format_prompt(eval_input: EvalInput) -> str:
    """Format the prompt for a single evaluation input."""
    prompt_variables = {
        "INPUTS": format_vars(eval_input.inputs),
        "OUTPUT": format_vars([eval_input.output]),
        "EVALUATION_CRITERIA": metric.criteria,
        "RUBRIC": format_rubric(metric.rubric),
    }
    return format_user_prompt(prompt_variables)

prompts = [format_prompt(eval_input) for eval_input in eval_inputs_batch]
prompts

['# GOAL\nYour job is to evaluate a task carried out by an AI system powered by a large language model.\n\nYou will be provided with the inputs and output of the task, as well as the evaluation criteria and scoring rubric. Your task is to evaluate the output of the AI system based on the evaluation criteria and scoring rubric provided.\n\n# INPUT\nBelow are the inputs required for performing the task:\n<inputs>\n<query>\nPlease read the technical issue that the user is facing and help me create a detailed solution based on the context provided.\n</query>\n<context>\n# Customer Issue: I\'m having trouble pushing large files to my Git repository. It says \'File size exceeds GitHub\'s file size limit of 100 MB\'.\n\n# Context: Git Large File Storage (LFS) replaces large files such as audio samples, videos, datasets, and graphics with text pointers inside Git, while storing the file contents on a remote server like GitHub.com or GitHub Enterprise.\n\nTo use Git LFS, you need to download an

In [5]:
from flow_judge.models.adapters.baseten.adapter import AsyncBasetenAPIAdapter

adapter = AsyncBasetenAPIAdapter(
    model_id="<your_model_id>",
    batch_size=2,
    webhook_proxy_url="https://proxy.flowrite.com"
)

await adapter._async_fetch_batched_response(prompts)

2024-10-17 17:59:03 [debug    ] Batch 0: [{'index': 1, 'content': '# GOAL\nYour job is to evaluate a task carried out by an AI system powered by a large language model.\n\nYou will be provided with the inputs and output of the task, as well as the evaluation criteria and scoring rubric. Your task is to evaluate the output of the AI system based on the evaluation criteria and scoring rubric provided.\n\n# INPUT\nBelow are the inputs required for performing the task:\n<inputs>\n<query>\nPlease read the technical issue that the user is facing and help me create a detailed solution based on the context provided.\n</query>\n<context>\n# Customer Issue: I\'m having trouble pushing large files to my Git repository. It says \'File size exceeds GitHub\'s file size limit of 100 MB\'.\n\n# Context: Git Large File Storage (LFS) replaces large files such as audio samples, videos, datasets, and graphics with text pointers inside Git, while storing the file contents on a remote server like GitHub.com

[[{'request_id': '66626438619db84822a57078a81e0e99',
   'response': '<feedback>\nThe response is mostly consistent with the provided context, with only minor and inconsequential inconsistencies or fabrications. The response correctly identifies the need to use Git Large File Storage (LFS) to resolve the issue of pushing large files to a Git repository, which aligns with the context provided. It also accurately outlines the steps to install and set up Git LFS, which are supported by the context.\n\nHowever, there are a few minor issues:\n1. The response suggests using `*.large-file-extension` to track large files, but the context does not provide specific file extensions. Instead, it mentions using "*.large-file-extension" as a placeholder, which is a slight deviation from the context.\n2. The response mentions committing the large files using `git add large-file.ext`, but the context does not provide specific file names or extensions. This is a minor inconsistency as the response is st